In [1]:
from src.constants import places
import requests
import pprint
import re
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import os
import json

In [2]:
from pymongo import MongoClient

client = MongoClient()

def connectCollection(database, collection):
    ''' return collection from db crunchbase'''
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection('dbcompanies','companies_clean')

In [4]:
companies = db.companies_clean.find({ 
    "money_raised":{
        "$gte": 1000000
    }}) 

In [5]:
data_companies = pd.DataFrame(companies)

In [6]:
geo_point = []
for loc in range(len(data_companies)):
    geo_point.append(data_companies.coordinates[loc])  

In [7]:
def near(geo_point, radio_meters):
        geopoint = geo_point
        return list(db.companies_clean.find({
        "coordinates.coordinates": {
         "$near": {
             "$geometry": geopoint,
             "$maxDistance": radio_meters
         }}})) 

radio_meters = 3000
number_offices = []
list_offices=[]
for i in range(len(data_companies)):
    num_offices =  near(geo_point[i], radio_meters)
    list_offices.append(num_offices)
    number_offices.append(len(num_offices))
data_companies['number offices near'] = number_offices

In [8]:
data_companies = data_companies[data_companies['number offices near'] > 1]

In [9]:
data_companies.head()

,_id,name,employees,year,category,id,money_raised,city,country,latitude,longitude,coordinates,number offices near
0,5e049e9de6eb90cad2d707ad,Fixya,30.0,2013,web,52cdef7c4bab8bd675297fec,8000000.0,San Mateo,USA,37.566879,-122.323895,"{'type': 'Point', 'coordinates': [-122.323895,...",2
2,5e049e9de6eb90cad2d707af,Social Gaming Network,100.0,2011,web,52cdef7c4bab8bd67529831a,17100000.0,Los Angeles,USA,37.446823,-122.161523,"{'type': 'Point', 'coordinates': [-122.161523,...",3
3,5e049e9de6eb90cad2d707b0,Skydeck,9.0,2012,web,52cdef7c4bab8bd675298509,4000000.0,Palo Alto,USA,37.564538,-122.325470,"{'type': 'Point', 'coordinates': [-122.3254705...",2
4,5e049e9de6eb90cad2d707b1,PeekYou,20.0,2012,web,52cdef7c4bab8bd675297f94,1830000.0,New York,USA,40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ...",6
9,5e049e9de6eb90cad2d707b7,Inuvo,50.0,2009,web,52cdef7d4bab8bd67529980b,4200000.0,Clearwater,USA,27.913011,-82.702449,"{'type': 'Point', 'coordinates': [-82.702449, ...",2


In [10]:
geo = []
for coords in data_companies['coordinates']:
    geo.append(coords['coordinates'])

In [11]:
 # Latitud 37
# Longitud -122
geo

[[-122.323895, 37.566879],
 [-122.161523, 37.446823],
 [-122.3254705, 37.5645375],
 [-73.985506, 40.757929],
 [-82.702449, 27.913011],
 [18.036587, 59.337864],
 [18.036587, 59.337864],
 [-82.702449, 27.913011],
 [-122.397582, 37.785271],
 [-0.1020265, 51.5196135],
 [-73.987764, 40.744618],
 [-0.0936688, 51.5119418],
 [-73.9822211, 40.7454098],
 [-74.0136605, 40.7078343],
 [-73.994797, 40.727216],
 [-118.489748, 34.007112],
 [-73.990967, 40.73343],
 [-74.011114, 40.707954],
 [-118.489681, 34.007967],
 [-122.419209, 37.775206],
 [4.8928722, 52.3657578],
 [-74.00717, 40.7408042]]

In [166]:
def getNearby(lat,lng,query,radius):
        CLIENT_ID = os.getenv("client_id")
        CLIENT_SECRET = os.getenv("client_secret")
        url = 'https://api.foursquare.com/v2/venues/explore'
        queryParams = dict(
            client_id=CLIENT_ID,
            client_secret=CLIENT_SECRET,
            v='20181023', # DONDE ESTOY
            ll=f"{lat},{lng}", #QUE QUIERO
            query=query, # A QUE DISTANCIA
            radius=radius,
            limit = 25
        )
        res = requests.get(url, params=queryParams)
        return res.json()
  

In [167]:
# Lo comento porque al hacer Run, no quiero volver a hacer las requests
p = []
for coord in geo:
    for place, value in places.items():
        try:
            request = (getNearby(coord[1],coord[0],value['api'],value['distance']))
            p.append(request)
        except:
            Exception: f"Ha habido problemas con {place}"


In [168]:
def curateResponse(data):
    places = []
    for item in data:
        try:
            for i in item["response"]["groups"][0]["items"]:
                if i["venue"]['location']['city'] != None:
                    places.append({
                        "name":i["venue"]['name'],
                        "distance": i["venue"]['location']["distance"],
                        "country": i["venue"]['location']["country"],
                        "city": i["venue"]['location']['city'],
                        "latitude": i["venue"]['location']["lat"],
                        "longitude": i["venue"]['location']["lng"],
                        "coordinates": {
                        "type":"Point",
                        "coordinates":[i["venue"]['location']["lng"], i["venue"]['location']["lat"]]
                            }})
        except:
            print(f"Ha ocurrido algún error.")
    return places

In [169]:
data = pd.DataFrame(curateResponse(p))

Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.


In [170]:
data.shape

(1791, 7)

In [171]:
data.name.unique()

array(['Starbucks',
       'San Francisco International Airport (SFO) (San Francisco International Airport)',
       'San Carlos Airport (SQL) (San Carlos Airport)',
       'The Centurion Lounge', 'Terminal 2', 'Harvey Milk Terminal 1',
       'International Terminal', 'Boarding Area B', 'Boarding Area C',
       'Boarding Area D', 'International Terminal A', 'Boarding Area E',
       'TSA Pre Check Terminal 3', 'Terminal 3', 'Main Hall',
       'Boarding Area F', 'International Terminal G',
       'Alaska Airlines Flight 741', 'Gate A15',
       'jetBlue Airways Ticket Counter', 'TSA Security Checkpoint',
       'Remote Spot 9.3', 'Gate B17', 'Gate B13', 'Gate B14', 'Gate B12',
       'TreeRing', 'San Mateo Nippon Gakuen',
       "St. Matthew's Episcopal Day School",
       'Kumon Math and Reading Center of San Mateo - Hillsborough',
       'Kinder Academy Montessori', 'the Coder School',
       'Centennial Montessori School', 'North Shoreview Montessori',
       'Stanbridge Academy',

In [187]:
found = data[data['name'].str.contains('Vegan')]

In [188]:
found.name.value_counts()

Vegan Junk Food Bar           2
Garden Fresh Vegan Cuisine    2
Elovate Vegan Kitchen         2
Thai Vegan                    2
MTHR Vegan                    1
Maoz Vegan                    1
Urban Vegan Kitchen           1
Golden Era Vegan              1
Name: name, dtype: int64

In [175]:
data.name.value_counts().head()

Starbucks                   360
sweetgreen                   14
Abokado                      11
Juice Press                  11
Corporate Express Travel      8
Name: name, dtype: int64

In [192]:
# Vamos a organizar los datos, y solo nos quedaremos en la columna 'type' con los valores 
# del diccionario <groups> 
groups= {
    "Coffee Shop": 'Starbucks',
    "Airport":['San Carlos Airport (SQL) (San Carlos Airport)',
                "Santa Monica Airport (SMO) (Santa Monica Airport)","London City Airport (LCY) (London City Airport)",
                "St. Petersburg - Clearwater International Airport (PIE)",
                "Stockholm-Bromma Airport (BMA) (Stockholm-Bromma Airport)",
                "San Francisco International Airport (SFO) (San Francisco International Airport)",
                "LaGuardia Airport (LGA) (LaGuardia Airport)"],
    "Bar":["Bar","Nightcub","Cocktail Bar","Pub","Plunge Rooftop Bar & Lounge","Audio Nightclub","Temple Nightclub",
            "Momofuku Ssäm Bar","Absinthe Brasserie & Bar","Cue Bar","Ritz Bar & Lounge"],
    "School": ["Petite Sorbonne Preschool","Trinity Preschool","Preschool of America","School","MS 297 Middle School",
               "Village Community School","Chelsea School","Public School 3",
              "De School Van Mieke Petiet","Goethe Instituut","Altra College","AltSchool",
               "Alta Vista School","Mission Montessori","Step by Step Early Childhood",
               "Grace Church School","Greenwich Village School","Santa Monica High School",
               "Stuyvesant High School","Norman Thomas High School","North Shoreview Montessori"],
    "Vegan": ["Thai Vegan ","Maoz Vegan" , "Urban Vegan Kitchen", "Golden Era Vegan", 
              "Vegan Junk Food Bar","Garden Fresh Vegan Cuisine ","Elovate Vegan Kitchen"]
}
def chooseCat(prod):
    for groupName, groupItems in groups.items():
        if prod in groupItems:
            return groupName
    return "OTHER"

In [193]:
data["category"] = data.name.apply(chooseCat)

In [194]:
data.category.value_counts()

OTHER          1376
Coffee Shop     360
School           26
Airport          12
Bar              10
Vegan             7
Name: category, dtype: int64

In [ ]:
df = df[df.category != 'OTHER']

In [ ]:
data = df.reset_index(drop=True)

In [ ]:
import json
data.to_json('./queries.json', orient='records',default_handler=str)

In [ ]:
# Voy a intentar unir los dos DF: el de las compañias y el de las quieries:
result = pd.concat([data_companies, data])

In [ ]:
result.country.unique()

In [ ]:
result.loc[(result['country']== 'United States'), 'country'] = 'USA'
result.loc[(result['country']== 'Sverige'), 'country'] = 'SWE'
result.loc[(result['country']== 'United Kingdom'), 'country']  = 'GBR'
result.loc[(result['country']== 'Nederland'), 'country']  = 'NLD'


In [ ]:
result.country.unique()

In [ ]:
new_data = result[["coordinates", "country", "category", "name"]]

In [ ]:
new_data.head(-10)

In [ ]:
new_data.to_json('./companies_and_queries.json', orient='records',default_handler=str)